modeling.ipynb 에서 추가재학습하는파일

In [2]:
import os
import shutil
import yaml
from glob import glob
from ultralytics import YOLO
from roboflow import Roboflow
import pandas as pd

# Roboflow API 키 설정 (본인의 키 사용을 권장합니다)
ROBOFLOW_API_KEY = "NjIXpou4o4gsuGClT8hI"

# 기본 경로 설정
BASE_DATA_PATH = os.getcwd()
print(f"프로젝트 기본 경로: {BASE_DATA_PATH}")

프로젝트 기본 경로: c:\Users\Admin\work space\2nd


In [3]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# --- 기존 데이터셋 ---
# 데이터셋 1: acne-kbm0q
project1 = rf.workspace("kritsakorn").project("acne-kbm0q")
dataset1 = project1.version(21).download("yolov11")
print(f"데이터셋 1 다운로드 완료: {dataset1.location}")

# 데이터셋 2: whitehead-kavgp
project2 = rf.workspace("test-l2t0m").project("whitehead-kavgp")
dataset2 = project2.version(1).download("yolov11")
print(f"데이터셋 2 다운로드 완료: {dataset2.location}")

# 데이터셋 3: lesion-detection-ruc40
project3 = rf.workspace("skinlesions").project("lesion-detection-ruc40")
dataset3 = project3.version(10).download("yolov11")
print(f"데이터셋 3 다운로드 완료: {dataset3.location}")

# --- 새로 추가할 데이터셋 ---
# 데이터셋 4: acne-gvhhe-bcjln
project4 = rf.workspace("test-l2t0m").project("acne-gvhhe-bcjln")
dataset4 = project4.version(1).download("yolov11")
print(f"새 데이터셋 다운로드 완료: {dataset4.location}")

loading Roboflow workspace...
loading Roboflow project...
데이터셋 1 다운로드 완료: c:\Users\Admin\work space\2nd\Acne-21
loading Roboflow workspace...
loading Roboflow project...
데이터셋 2 다운로드 완료: c:\Users\Admin\work space\2nd\whitehead-1
loading Roboflow workspace...
loading Roboflow project...
데이터셋 3 다운로드 완료: c:\Users\Admin\work space\2nd\lesion-detection-10
loading Roboflow workspace...
loading Roboflow project...
새 데이터셋 다운로드 완료: c:\Users\Admin\work space\2nd\Acne-1


In [4]:
# --- 다운로드된 폴더 이름 설정 (실제 생성된 이름과 다를 경우 수정) ---
DATASET_1_NAME = 'Acne-21'
DATASET_2_NAME = 'whitehead-1'
DATASET_3_NAME = 'lesion-detection-10'
DATASET_4_NAME = 'Acne-1' # 새로 추가된 데이터셋 폴더명

# --- 최종 결과물 폴더 이름 설정 ---
FINAL_DATASET_NAME = 'skin_lesion_final_combined'

# --- 각 데이터셋의 클래스 목록 정의 ---
CLASS_NAMES_1 = ['Acne', 'comedone', 'cyst', 'nodule', 'papule', 'pustule', 'whitehead']
CLASS_NAMES_2 = ['whitehead']
CLASS_NAMES_3 = ['acne', 'eczema', 'psoriasis', 'rosacea']
CLASS_NAMES_4 = ['cyst', 'nodule', 'comedone', 'papule', 'pustule']

# --- 모든 클래스 목록 통합 및 정규화 ---
# 이름 정규화 ('whitehead' -> 'whiteheads', 'acne' -> 'Acne')
normalized_names_1 = [name.replace('whitehead', 'whiteheads') for name in CLASS_NAMES_1]
normalized_names_2 = [name.replace('whitehead', 'whiteheads') for name in CLASS_NAMES_2]
normalized_names_3 = [name.replace('acne', 'Acne') for name in CLASS_NAMES_3]

# 모든 클래스를 set으로 합쳐 중복 제거 후 리스트로 변환 및 정렬
final_class_set = set(normalized_names_1) | set(normalized_names_2) | set(normalized_names_3) | set(CLASS_NAMES_4)
FINAL_CLASS_NAMES = sorted(list(final_class_set))

print(f"✅ 최종 통합 클래스 목록 (총 {len(FINAL_CLASS_NAMES)}개):")
print(FINAL_CLASS_NAMES)

✅ 최종 통합 클래스 목록 (총 10개):
['Acne', 'comedone', 'cyst', 'eczema', 'nodule', 'papule', 'psoriasis', 'pustule', 'rosacea', 'whiteheads']


In [5]:
# --- 최종 클래스 이름 -> ID 딕셔너리 생성 ---
final_name_to_id = {name: i for i, name in enumerate(FINAL_CLASS_NAMES)}

# --- 각 데이터셋별 재매핑 테이블 생성 ---
remap_1 = {i: final_name_to_id[name.replace('whitehead', 'whiteheads')] for i, name in enumerate(CLASS_NAMES_1)}
remap_2 = {i: final_name_to_id[name.replace('whitehead', 'whiteheads')] for i, name in enumerate(CLASS_NAMES_2)}
remap_3 = {i: final_name_to_id[name.replace('acne', 'Acne')] for i, name in enumerate(CLASS_NAMES_3) if name.replace('acne', 'Acne') in final_name_to_id}
remap_4 = {i: final_name_to_id[name] for i, name in enumerate(CLASS_NAMES_4)}

print("✅ ID 재매핑 테이블 생성 완료!")

# --- 데이터 병합을 위한 함수 정의 ---
def process_dataset(src_path, remap_dict, final_path):
    if not os.path.exists(src_path):
        print(f"⚠️ 경고: 소스 경로를 찾을 수 없습니다: {src_path}. 건너뜁니다.")
        return
    print(f"'{os.path.basename(src_path)}' 데이터셋 처리 중...")
    for split in ['train', 'valid', 'test']:
        src_split_path = os.path.join(src_path, split)
        if not os.path.exists(src_split_path): continue
        img_files = glob(os.path.join(src_split_path, 'images', '*.jpg')) + \
                    glob(os.path.join(src_split_path, 'images', '*.jpeg')) + \
                    glob(os.path.join(src_split_path, 'images', '*.png'))
        for img_file in img_files:
            shutil.copy(img_file, os.path.join(final_path, split, 'images'))
            base_filename = os.path.splitext(os.path.basename(img_file))[0]
            lbl_file = os.path.join(src_split_path, 'labels', f"{base_filename}.txt")
            if os.path.exists(lbl_file):
                new_lbl_content = []
                with open(lbl_file, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if not parts: continue
                        original_id = int(parts[0])
                        if original_id in remap_dict:
                            new_id = remap_dict[original_id]
                            new_line = f"{new_id} {' '.join(parts[1:])}"
                            new_lbl_content.append(new_line)
                if new_lbl_content:
                    new_lbl_path = os.path.join(final_path, split, 'labels', f"{base_filename}.txt")
                    with open(new_lbl_path, 'w') as f:
                        f.write('\n'.join(new_lbl_content))

✅ ID 재매핑 테이블 생성 완료!


In [6]:
# --- 최종 데이터셋 폴더 생성 ---
final_path = os.path.join(BASE_DATA_PATH, FINAL_DATASET_NAME)
if os.path.exists(final_path):
    shutil.rmtree(final_path)
for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(final_path, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(final_path, split, 'labels'), exist_ok=True)
print(f"'{final_path}' 폴더 구조를 생성했습니다.")

# --- 모든 데이터셋 병합 실행 ---
process_dataset(os.path.join(BASE_DATA_PATH, DATASET_1_NAME), remap_1, final_path)
process_dataset(os.path.join(BASE_DATA_PATH, DATASET_2_NAME), remap_2, final_path)
process_dataset(os.path.join(BASE_DATA_PATH, DATASET_3_NAME), remap_3, final_path)
process_dataset(os.path.join(BASE_DATA_PATH, DATASET_4_NAME), remap_4, final_path)
print("-" * 50)
print("✅ 모든 데이터 병합 및 라벨 재매핑 완료!")

# --- 최종 data.yaml 파일 생성 ---
final_yaml_path = os.path.join(final_path, 'data.yaml')
yaml_data = {
    'path': os.path.abspath(final_path),
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': len(FINAL_CLASS_NAMES),
    'names': FINAL_CLASS_NAMES
}
with open(final_yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(yaml_data, f, allow_unicode=True, sort_keys=False)
print(f"✅ '{final_yaml_path}' 파일 생성 완료!")

'c:\Users\Admin\work space\2nd\skin_lesion_final_combined' 폴더 구조를 생성했습니다.
'Acne-21' 데이터셋 처리 중...
'whitehead-1' 데이터셋 처리 중...
'lesion-detection-10' 데이터셋 처리 중...
'Acne-1' 데이터셋 처리 중...
--------------------------------------------------
✅ 모든 데이터 병합 및 라벨 재매핑 완료!
✅ 'c:\Users\Admin\work space\2nd\skin_lesion_final_combined\data.yaml' 파일 생성 완료!


In [7]:
# --- 모델 학습 ---
# 처음부터 학습하는 것을 추천 (yolov8s.pt 사용)
model = YOLO('yolov8s.pt')

print("🚀 모델 재학습을 시작합니다...")
results = model.train(
    data=final_yaml_path,
    epochs=100,
    imgsz=640,
    batch=8,
    project=BASE_DATA_PATH,
    name='skin_lesion_final_model', # 최종 결과 폴더 이름
    exist_ok=True,
    patience=10 # 10 에포크 동안 성능 향상 없으면 조기 종료
)
print("\n🎉 재학습이 완료되었습니다!")

# --- 모델 평가 ---
print("\n🚀 최종 모델의 성능을 테스트 데이터셋으로 검증합니다...")
best_model_path = os.path.join(results.save_dir, 'weights', 'best.pt')
best_model = YOLO(best_model_path)
metrics = best_model.val(data=final_yaml_path, split='test')

print("\n--- ✅ 최종 성능 지표 ---")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")

🚀 모델 재학습을 시작합니다...
New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Admin\work space\2nd\skin_lesion_final_combined\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0

KeyboardInterrupt: 